In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

spark = (
    SparkSession.builder
    .appName("Airlines BI Queries")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

dim_marketing = spark.read.csv("../output/Dim_Marketing_Airline.csv", header=True, inferSchema=True)
dim_operating = spark.read.csv("../output/Dim_Operating_Airline.csv", header=True, inferSchema=True)
dim_airport   = spark.read.csv("../output/Dim_Airport.csv", header=True, inferSchema=True)
dim_time      = spark.read.csv("../output/Dim_Time.csv", header=True, inferSchema=True)
dim_date      = spark.read.csv("../output/Dim_Date.csv", header=True, inferSchema=True)
fact_flight   = spark.read.csv("../output/FactFlight.csv", header=True, inferSchema=True)

fact_flight.printSchema()
dim_date.printSchema()


root
 |-- flight_id: string (nullable = true)
 |-- date_id: string (nullable = true)
 |-- departure_time_id: string (nullable = true)
 |-- arrival_time_id: string (nullable = true)
 |-- origin_airport_id: string (nullable = true)
 |-- destination_airport_id: string (nullable = true)
 |-- marketing_airline_id: string (nullable = true)
 |-- operating_airline_id: string (nullable = true)
 |-- tail_number: string (nullable = true)
 |-- dep_delay_minutes: double (nullable = true)
 |-- arr_delay_minutes: double (nullable = true)
 |-- crs_elapsed_time: double (nullable = true)
 |-- actual_elapsed_time: double (nullable = true)
 |-- distance: double (nullable = true)
 |-- status: string (nullable = true)
 |-- flight_complexity_score: double (nullable = true)

root
 |-- date_id: string (nullable = true)
 |-- flight_date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day_of_month: integer (nullable =

In [ ]:
delta_flights_operating = (
    fact_flight.alias("f")
    .join(dim_operating.alias("o"), F.col("f.operating_airline_id") == F.col("o.operating_airline_id"), "inner")
    .filter(F.col("o.airline_name").like("%Delta Air Lines%"))
    .select(F.col("f.flight_id"))
    .distinct()  
    .count()
)

print(f"Кількість рейсів, виконаних Delta Air Lines (з Dim_Operating_Airline): {delta_flights_operating}")

Кількість рейсів, виконаних Delta Air Lines (з Dim_Operating_Airline): 3294917


In [4]:
delayed_n123aa_2021 = (
    fact_flight.alias("f")
    .join(dim_date.alias("d"), F.col("f.date_id") == F.col("d.date_id"), "inner")
    .filter((F.col("d.year") == 2021) &
            (F.col("f.tail_number") == "N789SK") &
            (F.col("f.status") == "delayed"))
    .select(
    F.col("f.flight_id"),
    F.col("d.flight_date"),
    F.col("f.tail_number"),
    F.col("f.status"),
    F.col("f.dep_delay_minutes"),
    F.col("f.arr_delay_minutes"),
    F.col("f.origin_airport_id"),
    F.col("f.destination_airport_id")
)
)

delayed_n123aa_2021.show(20, truncate=False)

+------------------------------------+-----------+-----------+-------+-----------------+-----------------+------------------------------------+------------------------------------+
|flight_id                           |flight_date|tail_number|status |dep_delay_minutes|arr_delay_minutes|origin_airport_id                   |destination_airport_id              |
+------------------------------------+-----------+-----------+-------+-----------------+-----------------+------------------------------------+------------------------------------+
|c8b5cd28-ed96-488e-962e-cb2af4f69ae3|2021-03-18 |N789SK     |delayed|176.0            |181.0            |819148d4-74bc-42d4-ac5b-f0a411c085a8|f7b344b0-e1ef-4897-b4a3-222516171479|
|52619af3-c500-4287-a599-95459b0af30e|2021-03-18 |N789SK     |delayed|208.0            |195.0            |4eb3f436-7cd4-422d-87a9-cd5d254cb8da|819148d4-74bc-42d4-ac5b-f0a411c085a8|
|8da19a83-9880-4c8e-8ccd-0041df25c724|2021-03-20 |N789SK     |delayed|90.0             |86.0   

In [ ]:
from pyspark.sql import functions as F

ca_states = ["ca", "california"]

ca_delays_by_airport = (
    fact_flight.alias("f")
    .join(
        dim_airport.alias("a"),
        F.col("f.origin_airport_id") == F.col("a.airport_id"),
        "inner"
    )
    .withColumn("state_lc", F.trim(F.lower(F.col("a.state_name"))))
    .filter(F.col("state_lc").isin(*ca_states))
    .filter(
        (F.col("f.status") == "delayed")
        | (F.col("f.dep_delay_minutes") > 0)
        | (F.col("f.arr_delay_minutes") > 0)
    )
    .withColumn("city", F.coalesce(F.col("a.city_name"), F.col("a.airport_code")))
    .groupBy(
        F.col("a.airport_id").alias("airport_id"),
        F.col("a.airport_code").alias("iata"),
        F.col("city")
    )
    .agg(F.count("*").alias("delay_count"))
    .orderBy(F.col("delay_count").desc())
)

print("Топ-20 аеропортів Каліфорнії за кількістю затриманих рейсів (аеропорти вильоту):")
ca_delays_by_airport.show(20, truncate=False)

ca_delays_by_city = (
    ca_delays_by_airport
    .groupBy(F.col("city"))
    .agg(F.sum("delay_count").alias("delay_count"))
    .orderBy(F.col("delay_count").desc())
)

print("Топ-20 міст Каліфорнії за кількістю затриманих рейсів (вильоти, агреговано за містом):")
ca_delays_by_city.show(20, truncate=False)


Топ-20 аеропортів Каліфорнії за кількістю затриманих рейсів (аеропорти вильоту):
+------------------------------------+----+-------------------+-----------+
|airport_id                          |iata|city               |delay_count|
+------------------------------------+----+-------------------+-----------+
|a9e8ac85-fab9-4770-b0e2-0a17aa406e55|LAX |Los Angeles, CA    |360455     |
|86c65dfd-7fe2-418d-b727-e8df1285aa93|SFO |San Francisco, CA  |243658     |
|89e42a4b-d352-4720-bf0f-5ce0fc3b5e60|SAN |San Diego, CA      |140205     |
|7e430d3c-256a-4557-bd93-47b85c94d78f|OAK |Oakland, CA        |91422      |
|bdec619e-c0ad-41db-a9b7-f4bd7ef28997|SJC |San Jose, CA       |87039      |
|e8fc7486-c71b-4f4c-80c9-47c53d14243c|SMF |Sacramento, CA     |84886      |
|031c7f8f-0dff-405b-ab59-a4cc88b0fcf6|SNA |Santa Ana, CA      |69292      |
|52639e6c-68ce-48ea-bd6e-fe1ba9137503|BUR |Burbank, CA        |46478      |
|e21e34d5-991b-470e-9824-ee9878bbcd9f|ONT |Ontario, CA        |36851      |
|39f78f

In [ ]:
from pyspark.sql import functions as F

atl_origin_airports = (
    dim_airport
    .withColumn("city_lc", F.lower(F.col("city_name")))
    .filter(
        (F.col("city_lc").like("%atlanta%")) |
        (F.col("airport_code") == "ATL")
    )
    .select(F.col("airport_id").alias("origin_airport_id"))
    .distinct()
)

atl_to_cities_2022 = (
    fact_flight.alias("f")
    .join(dim_date.alias("d"), F.col("f.date_id") == F.col("d.date_id"), "inner")
    .join(atl_origin_airports.alias("ao"), F.col("f.origin_airport_id") == F.col("ao.origin_airport_id"), "inner")
    .join(dim_airport.alias("dest"), F.col("f.destination_airport_id") == F.col("dest.airport_id"), "left")
    .filter(F.col("d.year") == 2022)
    .withColumn("dest_city", F.coalesce(F.col("dest.city_name"), F.col("dest.airport_code")))
)

atl_dest_city_stats = (
    atl_to_cities_2022
    .groupBy("dest_city")
    .agg(
        F.count("*").alias("flight_count"),
        F.countDistinct(F.col("f.destination_airport_id")).alias("distinct_destination_airports")
    )
    .orderBy(F.col("flight_count").desc())
)

print("Топ-20 міст призначення з прямими рейсами з Атланти у 2022 році (за кількістю рейсів):")
atl_dest_city_stats.show(20, truncate=False)

Топ-20 міст призначення з прямими рейсами з Атланти у 2022 році (за кількістю рейсів):
+---------------------+------------+-----------------------------+
|dest_city            |flight_count|distinct_destination_airports|
+---------------------+------------+-----------------------------+
|New York, NY         |6875        |2                            |
|Washington, DC       |5784        |2                            |
|Chicago, IL          |4653        |2                            |
|Orlando, FL          |4520        |1                            |
|Fort Lauderdale, FL  |4126        |1                            |
|Miami, FL            |3996        |1                            |
|Houston, TX          |3809        |2                            |
|Newark, NJ           |3624        |1                            |
|Tampa, FL            |3423        |1                            |
|Dallas/Fort Worth, TX|3165        |1                            |
|Denver, CO           |3112        |1     

In [ ]:
avg_dep_delay_by_quarter = (
    fact_flight.alias("f")
    .join(dim_date.alias("d"), F.col("f.date_id") == F.col("d.date_id"), "inner")
    .groupBy(F.col("d.year").alias("year"), F.col("d.quarter").alias("quarter"))
    .agg(
        F.round(F.avg(F.col("f.dep_delay_minutes")), 2).alias("avg_dep_delay_minutes"),
        F.count(F.col("f.dep_delay_minutes")).alias("count_values")  
    )
    .orderBy(F.col("year"), F.col("quarter"))
)

print("Середня затримка вильоту (у хвилинах) за роками та кварталами:")
avg_dep_delay_by_quarter.show(200, truncate=False)


Середня затримка вильоту (у хвилинах) за роками та кварталами:
+----+-------+---------------------+------------+
|year|quarter|avg_dep_delay_minutes|count_values|
+----+-------+---------------------+------------+
|2018|1      |13.01                |1045597     |
|2018|2      |14.58                |1137662     |
|2018|3      |14.82                |1482586     |
|2018|4      |12.02                |1939054     |
|2019|1      |14.17                |1861152     |
|2019|2      |15.94                |2014907     |
|2019|3      |14.59                |2059649     |
|2019|4      |12.44                |2010772     |
|2020|1      |9.49                 |1853542     |
|2020|2      |3.86                 |611199      |
|2020|3      |4.88                 |1105482     |
|2020|4      |5.72                 |1154283     |
|2021|1      |7.76                 |1167314     |
|2021|2      |12.4                 |1554932     |
|2021|3      |15.85                |1760039     |
|2021|4      |13.33                |1

In [ ]:
from pyspark.sql import Window
w_tail = Window.partitionBy(F.col("tail_number")).orderBy(F.to_date(F.col("flight_date")), F.col("flight_id"))

df_tail = (
    fact_flight.alias("f")
    .join(dim_date.alias("d"), F.col("f.date_id") == F.col("d.date_id"), "left")
    .select(
        F.col("f.flight_id").alias("flight_id"),
        F.col("d.flight_date").alias("flight_date"),
        F.col("f.tail_number").alias("tail_number"),
        F.col("f.dep_delay_minutes").alias("dep_delay_minutes"),
        F.col("f.arr_delay_minutes").alias("arr_delay_minutes"),
        F.col("f.origin_airport_id").alias("origin_airport_id"),
        F.col("f.destination_airport_id").alias("destination_airport_id")
    )
    .filter(F.col("tail_number").isNotNull())
)

df_tail_prev = (
    df_tail
    .withColumn("prev_dep_delay", F.lag(F.col("dep_delay_minutes")).over(w_tail))
    .withColumn("dep_delay_diff", F.when(F.col("prev_dep_delay").isNotNull(), F.col("dep_delay_minutes") - F.col("prev_dep_delay")).otherwise(None))
)

print("Приклади рейсів, для яких існує попередній рейс того ж літака (показано різницю у затримці):")
df_tail_prev.filter(F.col("prev_dep_delay").isNotNull()).orderBy(F.col("tail_number"), F.col("flight_date")).show(100, truncate=False)

Приклади рейсів, для яких існує попередній рейс того ж літака (показано різницю у затримці):
+------------------------------------+-----------+-----------+-----------------+-----------------+------------------------------------+------------------------------------+--------------+--------------+
|flight_id                           |flight_date|tail_number|dep_delay_minutes|arr_delay_minutes|origin_airport_id                   |destination_airport_id              |prev_dep_delay|dep_delay_diff|
+------------------------------------+-----------+-----------+-----------------+-----------------+------------------------------------+------------------------------------+--------------+--------------+
|8c3d639f-b1f4-410d-aadb-fc7a602f5d32|2022-01-30 |202NV      |158.0            |158.0            |50eb5bde-f2e7-4bb2-b27b-41daa78d9251|f7d8fa81-22ec-4290-a553-71eead733e5c|167.0         |-9.0          |
|d453543a-bb9e-4ad5-9fd2-974133c77a36|2022-01-31 |202NV      |81.0             |73.0           